In [1]:
import numpy as np
import dill as pickle
import aesara
import aesara.tensor as aet
from biogeme.database import Database
import pandas as pd
from biogeme_aesara import DatabaseShared, BetaShared

swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
db = DatabaseShared("swissmetro", swissmetro, choiceVar="CHOICE")
globals().update(db.variables)

# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
db.remove(exclude)
db.data["CHOICE"] -= 1 # set the first choice to 0
db.autoscale(variables=['TRAIN_CO', 'TRAIN_TT', 'CAR_CO', 'CAR_TT', 
    'SM_CO', 'SM_TT'], verbose=True)

scaling TRAIN_TT by 100.0
scaling TRAIN_CO by 100.0
scaling SM_TT by 100.0
scaling SM_CO by 100.0
scaling CAR_TT by 100.0
scaling CAR_CO by 100.0


In [2]:
class mnl_model:
    def __init__(self):
        b_cost = BetaShared("b_cost", 0., None, None, 0)
        b_time = BetaShared("b_time", 0., None, None, 0)
        asc_train = BetaShared("asc_train", 0., None, None, 0)
        asc_car = BetaShared("asc_car", 0., None, None, 0)
        asc_sm = BetaShared("asc_sm", 0., None, None, 0)
        
        self.y = CHOICE.y
        self.params = [b_cost(), b_time(), asc_train(), asc_car()]
        self.full_params = [b_cost(), b_time(), asc_train(), asc_car(), asc_sm()]

        # Definition of the utility functions
        U_1 = b_cost * TRAIN_CO.x + b_time * TRAIN_TT.x + asc_train
        U_2 = b_cost * SM_CO.x + b_time * SM_TT.x + asc_sm
        U_3 = b_cost * CAR_CO.x + b_time * CAR_TT.x + asc_car

        utility_vector = aet.concatenate([U_1, U_2, U_3], axis=1)

        # symbolic expression for computing the matrix of class-membership
        # probabilities
        self.p_y_given_x = aet.nnet.softmax(utility_vector, axis=1)

        # symbolic description of how to compute prediction as class whose
        # probability is maximal
        self.pred = aet.argmax(self.p_y_given_x, axis=1)

        self.L1 = aet.sum([b_cost(), b_time()])
        

def neg_loglikelihood(prob, y):
    nll = -aet.mean(aet.log(prob)[aet.arange(y.shape[0]), y])
    return nll


def full_loglikelihood(prob, y):
    ll = aet.sum(aet.log(prob)[aet.arange(y.shape[0]), y])
    return ll


def errors(pred, y):
    if y.ndim != pred.ndim:
        raise TypeError(
            'y should have the same shape as pred',
            ('y', y.type, 'pred', pred.type)
        )
    if y.dtype.startswith('int'):
        return aet.mean(aet.neq(pred, y))
    else:
        raise NotImplementedError()

class Adam:
    def __init__(self, params, b1=0.9, b2=0.999, epsilon=1e-8):
        zero = np.array(0.).astype(aesara.config.floatX)
        self.m = [aesara.shared(p.get_value() * zero) for p in params]
        self.v = [aesara.shared(p.get_value() * zero) for p in params]
        self._t_prev = aesara.shared(zero)
        self.b1 = b1
        self.b2 = b2
        self.epsilon = epsilon

    def update(self, cost, params, lr=0.0001):
        one = np.array(1.).astype(aesara.config.floatX)
        grads = aet.grad(cost, params)
        t_prev = self._t_prev
        
        updates = []
        m_prev = self.m
        v_prev = self.v
        t = t_prev + 1.
        
        a_t = aet.sqrt(one-self.b2**t)/(one-self.b1**t)
        for m, v, param, grad in zip(m_prev, v_prev, params, grads):
            m_t = self.b1 * m + (one-self.b1) * grad
            v_t = self.b2 * v + (one-self.b2) * grad**2
            g_t = a_t * m_t / (aet.sqrt(v_t) + self.epsilon)
            p_t = param - lr * g_t
            
            updates.append((m, m_t))
            updates.append((v, v_t))
            updates.append((param, p_t))
        
        updates.append((t_prev, t))
        return updates

class RMSProp:
    def __init__(self, params, rho=0.9, epsilon=1e-8, decay=0.):
        zero = np.array(0.).astype(aesara.config.floatX)
        self._a = [aesara.shared(p.get_value() * zero) for p in params]
        self.rho = rho
        self.epsilon = epsilon
        self.decay = decay

    def update(self, cost, params, lr=0.001):
        grads = aet.grad(cost, params)

        updates = []
        accumulators = self._a
        for a, param, grad in zip(accumulators, params, grads):
            a_t = self.rho * a + (1. - self.rho) * grad**2
            g_t = grad / (aet.sqrt(a_t) + self.epsilon)
            p_t = param - lr * g_t

            updates.append((a, a_t))
            updates.append((param, p_t))
        
        return updates

class MomentumSGD:
    def __init__(self, params, momentum=0.9, nesterov=False):
        zero = np.array(0.).astype(aesara.config.floatX)
        self._moments = [aesara.shared(p.get_value() * zero) for p in params]
        self.momentum = momentum
        self.nesterov = nesterov

    def update(self, cost, params, lr=0.001):
        grads = aet.grad(cost, params)

        updates = []
        moments = self._moments
        for m, param, grad in zip(moments, params, grads):
            velocity = self.momentum * m - lr * grad

            if self.nesterov:
                p_t = param + self.momentum * velocity - lr * grad
            else:
                p_t = param + velocity
            
            updates.append((m, velocity))
            updates.append((param, p_t))
        return updates

class SGD:
    def __init__(self, params=None):
        pass

    def update(self, cost, params, lr=0.00001):
        grads = aet.grad(cost, params)

        updates = []
        for param, grad in zip(params, grads):
            p_t = param - lr * grad
            updates.append((param, p_t))
        return updates

learning_rate = aet.scalar("learning_rate")
model = mnl_model()
cost = neg_loglikelihood(model.p_y_given_x, model.y) + 0.01*model.L1

opt = Adam(model.params)
updates = opt.update(cost=cost, params=model.params, lr=learning_rate)

loglikelihood_estimation = aesara.function(
    inputs=db.inputs() + [learning_rate],
    outputs=cost,
    updates=updates,
    on_unused_input="ignore",
)

loglikelihood = aesara.function(
    inputs=db.inputs(),
    outputs=full_loglikelihood(model.p_y_given_x, model.y),
    on_unused_input="ignore",
)

output_probabilities = aesara.function(
    inputs=db.inputs(),
    outputs=model.p_y_given_x,
    on_unused_input="ignore",
)

output_estimated_betas = aesara.function(
    inputs=db.inputs(),
    outputs=model.full_params,
    on_unused_input="ignore",
) 

In [3]:
batch_size = 32
n_batches = db.data["CHOICE"].shape[0] // batch_size
print("batches per epoch:", n_batches)
print("batch size", batch_size)

nll_best = np.inf
error_best = 1.
ll_best = -np.inf
patience = 8000
patience_increase = 2
validation_frequency = min(n_batches, patience/2)
print("validation frequency", validation_frequency)
max_epoch = 50000
done_looping = False
epoch = 0
null_ll = loglikelihood(*(db.input_data()))
while (epoch < max_epoch) and (not done_looping):
    epoch = epoch + 1
    for minibatch_index in range(n_batches):
        r = np.random.randint(0, n_batches)
        s = np.random.randint(0, batch_size)
        iter = (epoch - 1) * n_batches + minibatch_index
        if (iter/patience) < 0.6:
            if (iter/patience) < 0.1:
                lr = 0.005
            else:
                lr = 0.001
        else:
            lr = 0.0001
        nll = loglikelihood_estimation(
            *(db.input_data(r, batch_size, s)) + [lr])
        
        if (iter + 1) % validation_frequency == 0:
            ll = loglikelihood(*(db.input_data()))
            if ll > ll_best:
                if ll > (ll_best * 1.005):
                    patience = max(patience, iter * patience_increase)
                    estimated_betas = {}
                    for n, p in enumerate(model.full_params):
                        bb = output_estimated_betas(*(db.input_data()))
                        estimated_betas[p] = np.round(bb[n].tolist(), 6)
                    print(estimated_betas)
                    print((
                        "epoch {0}, minibatch {1}/{2}, LL: {3:6.6f}, "
                        "progress {4:5.2f}%").format(
                        epoch, minibatch_index + 1, n_batches, ll, iter/patience * 100
                    ))
                ll_best = ll
                epoch_best = epoch
                betas_best = output_estimated_betas(*(db.input_data()))
                with open('best_model.pkl', 'wb') as f:
                    pickle.dump(model, f)
                
        if patience <= iter:
            done_looping = True
            break

with open('best_model.pkl', 'rb') as f:
    model = pickle.load(f)

output_errors = aesara.function(
    inputs=db.inputs(),
    outputs=errors(model.pred, model.y),
    on_unused_input="ignore",
)
error_best = output_errors(*(db.input_data()))

print(
    (
        "Optimization complete with best validation score of {0:6.3f}%, "
        "with negative loglikelihood of {1:9.6f}, at epoch #{2}"
    ).format(error_best * 100., ll_best, epoch_best)
)

output_estimated_betas = aesara.function(
    inputs=db.inputs(),
    outputs=model.full_params,
    on_unused_input="ignore",
) 
estimated_betas = {}
for n, p in enumerate(model.full_params):
    estimated_betas[p] = np.round(betas_best[n].tolist(), 4)
print(estimated_betas)

batches per epoch: 211
batch size 32
validation frequency 211
{b_cost: 0.176436, b_time: -0.72008, asc_train: -0.55205, asc_car: -0.275178, asc_sm: 0.0}
epoch 1, minibatch 211/211, LL: -6573.204438, progress  2.62%
{b_cost: 0.225013, b_time: -1.358846, asc_train: -0.934934, asc_car: -0.412618, asc_sm: 0.0}
epoch 2, minibatch 211/211, LL: -6294.171312, progress  5.26%
{b_cost: 0.234944, b_time: -1.867785, asc_train: -1.03369, asc_car: -0.529747, asc_sm: 0.0}
epoch 3, minibatch 211/211, LL: -6231.754555, progress  7.90%
{b_cost: 0.210407, b_time: -2.284235, asc_train: -1.133056, asc_car: -0.570314, asc_sm: 0.0}
epoch 4, minibatch 211/211, LL: -6206.182972, progress 10.54%
{b_cost: 0.203439, b_time: -2.381459, asc_train: -1.147321, asc_car: -0.591919, asc_sm: 0.0}
epoch 5, minibatch 211/211, LL: -6202.971119, progress 13.18%
{b_cost: 0.214044, b_time: -2.466934, asc_train: -1.14379, asc_car: -0.606585, asc_sm: 0.0}
epoch 6, minibatch 211/211, LL: -6201.429587, progress 15.81%
{b_cost: 0.1

In [22]:
from scipy import linalg
np.set_printoptions(suppress=True)

def hessian(prob, y, params):
    grads = aet.grad(full_loglikelihood(prob, y), params)
    _h = aet.as_tensor_variable(np.zeros((len(grads), len(grads))))
    for i in range(len(grads)):
        _h = aet.set_subtensor(x=_h[i,:], y=aet.grad(grads[i], model.params, 
		    consider_constant=None, disconnected_inputs="ignore"))
    return _h

def bh(prob, y, params):
    grads = aet.grad(full_loglikelihood(prob, y), params)
    _bh = aet.outer(aet.as_tensor_variable(grads), aet.as_tensor_variable(grads).T)
    return _bh

def variance_covariance(h):
    return -linalg.pinv(np.nan_to_num(h))

def rob_variance_covariance(h, bhhh):
    varcovar = variance_covariance(h)
    return varcovar.dot(bhhh.dot(varcovar))

def t_test(stderr, params):
    return [p.eval()/s for p, s in zip(params, stderr)]

def std_error(h, params):
    varcovar = variance_covariance(h)
    stderr = []
    for i in range(len(params)):
        if varcovar[i, i] < 0:
            stderr.append(np.finfo(float).max)
        else:
            stderr.append(np.sqrt(varcovar[i, i]))
    return stderr

def correlation_matrix(h):
    varcovar = variance_covariance(h)
    d = np.diag(varcovar)
    if (d > 0).all():
        diag = np.diag(np.sqrt(d))
        diagInv = linalg.inv(diag)
        correlation = diagInv.dot(varcovar.dot(diagInv))
    else:
        correlation = np.full_like(varcovar, np.finfo(float).max)
    return correlation

def rob_std_error(h, bhhh, params):
    varcovar = variance_covariance(h)
    robust_varcovar = varcovar.dot(bhhh.dot(varcovar))
    robstderr = []
    for i in range(len(params)):
        if robust_varcovar[i, i] < 0:
            robstderr.append(np.finfo(float).max)
        else:
            robstderr.append(np.sqrt(robust_varcovar[i, i]))
    return robstderr

def rob_correlation_matrix(h, bhhh):
    robust_varcovar = rob_variance_covariance(h, bhhh)
    rd = np.diag(robust_varcovar)
    if (rd > 0).all():
        diag = np.diag(np.sqrt(rd))
        diagInv = linalg.inv(diag)
        robust_correlation = diagInv.dot(robust_varcovar.dot(diagInv))
    else:
        robust_correlation = np.full_like(robust_varcovar, np.finfo(float).max)
    return robust_correlation

H = aesara.function(
    inputs=db.inputs(),
    outputs=hessian(model.p_y_given_x, model.y, model.params),
    on_unused_input="ignore",
)

BHHH = aesara.function(
    inputs=db.inputs(),
    outputs=bh(model.p_y_given_x, model.y, model.params),
    on_unused_input="ignore",
)

hessians = H(*(db.input_data()))
bhhh = BHHH(*(db.input_data()))
print("Hessians\n", hessians)
print("BHHH\n", bhhh)

# Print parameters
print([param for param in model.params])
print([param.eval()*1. for param in model.params])
# Standard errors
stderr = std_error(hessians, model.params)
print("standard errors:\n", np.round(stderr, 6))

ttest = t_test(stderr, model.params)
print("t-test\n", np.round(ttest, 6))

# Robust standard errors
robstderr = rob_std_error(hessians, bhhh, model.params)
print("rob. standard errors:\n", np.round(robstderr, 6))

ttest = t_test(robstderr, model.params)
print("rob. t-test\n", np.round(ttest, 6))

print("correlation matrix\n", correlation_matrix(hessians))

Hessians
 [[-2106.05301544   -15.04348334    21.7719284    523.64346   ]
 [  -15.04348334   -12.77181711   -53.91756945   -22.31861411]
 [   21.7719284    -53.91756945  -777.81529269   228.57886113]
 [  523.64346      -22.31861411   228.57886113 -1262.73149407]]
BHHH
 [[5798.21651409 1007.34754969  653.12165767 3055.02786274]
 [1007.34754969  175.01055426  113.46946081  530.76231705]
 [ 653.12165767  113.46946081   73.56881184  344.12389691]
 [3055.02786274  530.76231705  344.12389691 1609.66656203]]
[b_cost, b_time, asc_train, asc_car]
[0.20147672252093407, -3.8721094252865784, -1.1847555494628468, -0.636721105403731]
standard errors:
 [0.024062 0.372237 0.047879 0.034082]
t-test
 [  8.373072 -10.402274 -24.744969 -18.681881]
rob. standard errors:
 [0.02722  1.32577  0.078521 0.005414]
rob. t-test
 [   7.401702   -2.92065   -15.088396 -117.599695]
correlation matrix
 [[ 1.         -0.27683221  0.24892839  0.40951735]
 [-0.27683221  1.         -0.63388097 -0.43528266]
 [ 0.24892839 -0.